In [ ]:
from config import setup
setup()

In [3]:
from docplex.cp.model import CpoModel
from docplex.cp.model import *

In [5]:
G = [[],[]]    # Notre graphe

X, A = G

X = [[],[],[]] # Ensemble des sommets du graphe

E, T, S = X

E = [] # Ensemble de sommets à évacuer
T = [] # Ensemble de sommets de transfert
S = [] # Ensemble de sommets séurisés

for k in E:
    
    i_k, A_k, w_k, p_k, s_k, d_k, h_k = k
    
    i_k = 0  # Identifiant d'un sommet à évacuer
    A_k = [] # Liste des sommets pour le chemin d'évacuation
    w_k = 0  # Nombre de personnes à évacuer
    p_k = 0  # Durée d'évacuation d'un sommet
    s_k = 0  # Date début évacuation d'un sommet
    d_k = 0  # Date à laquelle le sommet crame
    h_k = 0  # Taux d'évacuation d'un sommet
    
    for e in A_k:
        
        e = 0 # Identifiant du sommet parcouru dans le chemin d'évacuation

for k in T:
    
    k = 0  # Identifiant d'un sommet de transfert
    
for k in S:
    
    k = 0  # Identifiant d'un sommet sécurisé
    
for e in A:
    
    i_e, in_e, out_e, c_e, l_e, b_e = e
    
    i_e   = 0 # Identifiant de l'arc
    in_e  = 0 # Identifiant du sommet entrant
    out_e = 0 # Identifiant du sommet sortant
    c_e   = 0 # Capacité de l'arc en personnes par minute
    l_e   = 0 # Temps de traversée en minutes
    b_e   = 0 # Date limite d'évacuation en minute



In [ ]:
def geosafe_model(G, ...):
    '''
    Integer Matrix G.l: length path from i to j = G.l[i,j].
                        no path ij <--> G.l[i,j] = 0
    Integer Array  G.q: capacity of node i <--> G.q[i]
    Boolean Array  G.eps: node i is starting node <--> G.eps[i] = 1
    
    '''

    mdl = CpoModel(name='geosafe')
    
    s = mdl.integer_var_list(G.E.shape[0], "Date début évacuation du sommet ")
    
    h = mdl.integer_var_list(G.E.shape[0], "Taux d'évacuation du sommet ")
    
    
    
    
    
    
    
    
    
    
    
    
